In [125]:
#!pip install checklist

In [126]:
#!pip install simpletransformers

In [91]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score

In [92]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
train_data = pd.read_csv('/content/data/olid-train.csv')

data = data[['text','labels']]
train_data = train_data[['text','labels']]
data.head(3)

,text,labels
0,@USER @USER Who the hell does he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1


In [93]:
nlp = spacy.load('en_core_web_sm')

In [94]:
perturbed_data = list(nlp.pipe(data['text']))

In [95]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [96]:
np.random.seed(42)
ret = Perturb.perturb(data['text'], Perturb.add_typos)
ret.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails fly after proetst honouring killed antifa arti... URL via @USER URL']

In [97]:
typo_data = []
for i in range(len(ret.data)):
  typo_data.append(ret.data[i][1]) # tweet with typos added

typo_data[:3]

['@USER @USER Who the hell does he thinkh e is?',
 '#BREAKING. #Greece: Molotov cocktails fly after proetst honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges aaginst a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!']

In [98]:
data_typo = pd.DataFrame({'text':typo_data,'label':data['labels']})
data_typo.head()

,text,label
0,@USER @USER Who the hell does he thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca on hte first fucking fight cba,1


In [99]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [100]:
model.train_model(train_data)

  0%|          | 0/13240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(1655, 0.5140549184332441)

In [101]:
predictions, raw_outputs = model.predict(list(data_typo['text']))
pred, out = model.predict(list(data['text']))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [102]:
predicted_typo_labels = pd.DataFrame({'text':data_typo.text, 'labels': predictions})
predicted_typo_labels.head()

,text,labels
0,@USER @USER Who the hell does he thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca on hte first fucking fight cba,1


In [103]:
result, model_outputs, wrong_predictions = model.eval_model(data_typo, report_typo = classification_report)
result_d, model_outputs_d, wrong_predictions_d = model.eval_model(data, report_normaldata = classification_report)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [104]:
print(result['report_typo'])
print("\n")
print(result_d['report_normaldata'])

              precision    recall  f1-score   support

         0.0       0.74      0.90      0.81        50
         1.0       0.87      0.68      0.76        50

    accuracy                           0.79       100
   macro avg       0.80      0.79      0.79       100
weighted avg       0.80      0.79      0.79       100



              precision    recall  f1-score   support

         0.0       0.78      0.90      0.83        50
         1.0       0.88      0.74      0.80        50

    accuracy                           0.82       100
   macro avg       0.83      0.82      0.82       100
weighted avg       0.83      0.82      0.82       100



In [105]:
print(model_outputs[:5])

[[-1.57128906  0.52392578]
 [ 0.66699219  0.11987305]
 [-1.6640625   0.74072266]
 [ 0.39990234  0.55419922]
 [-1.67578125  0.72363281]]


In [106]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[45  5]
 [16 34]]


### Provide 3 examples when the model failed to assign the correct label after perturbation.

In [107]:
print(data['labels'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [108]:
print(predicted_typo_labels['labels'][:10])

0    1
1    0
2    1
3    1
4    1
5    1
6    0
7    1
8    0
9    1
Name: labels, dtype: int64


In [109]:
failed_texts=[]
failed_labels=[]
for i in range(len(data)):
  if predicted_typo_labels['labels'][i] != data['labels'][i]:
    failed_texts.append(predicted_typo_labels['text'][i])
    failed_labels.append(predicted_typo_labels['labels'][i])

In [110]:
failed_examples = pd.DataFrame({'text':failed_texts, 'labels': failed_labels})
failed_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails fly afte...,0
1,#Christian #America – If we go by #Trump’s exa...,0
2,#Democrats #Liberals you are being #threatened...,0
3,(cr1tikal voice) smef m yass cheeks,0
4,@USER oh fukc off 😂😂,0
5,"#Emmys Well, most of those e-xcrackheads are ...",0
6,#BoycottNike campaign must continue and grow ...,0
7,#ANTIFA are bedfellow swith The Democratic Soc...,0
8,@USER Antifa has TS level nifluence. It's scary.,0
9,"#ConfirmKavanugh now, stall tactics are DC cro...",0


# 6. Negation *(4.5 points)* 

In [111]:
perturbed_data = list(nlp.pipe(data['text']))

In [112]:
ret = Perturb.perturb(perturbed_data, Perturb.add_negation) # negated data
ret.data[:5]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [113]:
negated_text = []
for i in range(len(ret.data)):
  negated_text.append(ret.data[i][1])

negated_text[:5]

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba"]

In [114]:
neg_predictions, neg_raw_outputs = model.predict(negated_text)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [115]:
predicted_neg_labels = pd.DataFrame({'text':negated_text, 'labels': data['labels']})
predicted_neg_labels.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [116]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_neg_labels, neg_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [117]:
print(result['neg_report'])

              precision    recall  f1-score   support

         0.0       0.79      0.92      0.85        50
         1.0       0.90      0.76      0.83        50

    accuracy                           0.84       100
   macro avg       0.85      0.84      0.84       100
weighted avg       0.85      0.84      0.84       100



In [118]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[45  5]
 [16 34]]


### Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion?

In [119]:
predicted_neg_labels['text'][:10]

0     @USER @USER Who the hell doesn't he think he is?
1    #BREAKING. #Greece: Molotov cocktails don't fl...
2    #OrrinHatch I can’t believe this sexist , clue...
3    @USER @USER I'll use that one the next time im...
4    0-1 didn't lose my acca on the first fucking f...
5    #Bakersfield is not why we need gun control! S...
6    #Christian #America – If we go by #Trump’s exa...
7    @USER @USER @USER She is not the most disingen...
8    #Democrats #Liberals you are being #threatened...
9    699. Just didn't want to tell you you should H...
Name: text, dtype: object

In [120]:
predicted_neg_labels_df = pd.DataFrame({'text':negated_text, 'labels': neg_predictions})
predicted_neg_labels_df.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [121]:
failed_neg_texts=[]
failed_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] != data['labels'][i]:
    failed_neg_texts.append(predicted_neg_labels['text'][i])
    failed_neg_labels.append(predicted_neg_labels['labels'][i])

In [122]:
failed_neg_examples = pd.DataFrame({'text':failed_neg_texts, 'labels': failed_neg_labels})
failed_neg_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails don't fl...,1
1,#Christian #America – If we go by #Trump’s exa...,1
2,#BoycottNike campaign must not continue and gr...,1
3,#ANTIFA are not bedfellows with The Democratic...,1
4,@USER Antifa doesn't have TS level influence. ...,1
5,"#ConfirmKavanugh now, stall tactics are not DC...",1
6,#Cuckservative Traitors Are not Worse Than For...,1
7,"#NEWS Jeff Sessions: If you want more death, '...",1
8,#SesameStreet #BertandErnie the conversation a...,1
9,#FemiNazis are not Liberals so #MeToo only cou...,1


In [123]:
success_neg_texts=[]
success_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] == data['labels'][i]:
    success_neg_texts.append(predicted_neg_labels['text'][i])
    success_neg_labels.append(predicted_neg_labels['labels'][i])

In [124]:
successful_neg_examples = pd.DataFrame({'text':success_neg_texts, 'labels': success_neg_labels})
successful_neg_examples

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,"#OrrinHatch I can’t believe this sexist , clue...",1
2,@USER @USER I'll use that one the next time im...,1
3,0-1 didn't lose my acca on the first fucking f...,1
4,#Bakersfield is not why we need gun control! S...,1
...,...,...
79,@USER He is obviously getting not suspended. H...,0
80,#Canada - EXCLUSIVE: #Trudeau #Liberals don't ...,0
81,@USER @USER ...than why didn't you show us how...,0
82,@USER @USER @USER You don't have yet to answer...,0


## 7. Creating negated examples